In [ ]:
from functools import partial, wraps
from pprint import pprint
import simpy

In [ ]:
def trace(env: simpy.Environment, callback):
    def get_wrapper(env_step, callback):
        """Generate the wrapper for env.step()."""

        @wraps(env_step)
        def tracing_step():
            """Call *callback* for the next event if one exist before
            calling ``env.step()``."""
            if len(env._queue):
                t, prio, eid, event = env._queue[0]
                callback(t, prio, eid, event)
            return env_step()

        return tracing_step

    env.step = get_wrapper(env.step, callback)


def monitor(data, t, prio, eid, event):
    data.append((t, eid, type(event)))


def test_process1(env: simpy.Environment):
    yield env.timeout(1)


def test_process2(env: simpy.Environment):
    yield env.timeout(2)


def test_process3(env: simpy.Environment):
    yield env.timeout(6)


data = []

monitor_binded = partial(monitor, data)

env = simpy.Environment()
trace(env, monitor_binded)

p1 = env.process(test_process1(env))
p2 = env.process(test_process2(env))
p3 = env.process(test_process3(env))

env.run(until=p1 & p2 & p3)

pprint(data)
